In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/My\ Drive/Flask/*.py

'/content/drive/My Drive/Flask/app2.py'  '/content/drive/My Drive/Flask/cnn.py'
'/content/drive/My Drive/Flask/app.py'	 '/content/drive/My Drive/Flask/prediction.py'


In [3]:
!cat '/content/drive/My Drive/Flask/prediction.py'

import numpy as np
from PIL import Image
from tensorflow.keras.models import model_from_json
#from sklearn.preprocessing import LabelEncoder
from google.colab import drive
drive.mount('/content/drive')

def getPrediction(file_path):
    
    #classes = ['NonDemented','VeryMildDemented','MildDemented','ModerateDemented']
    #le = LabelEncoder()
    #le.fit(classes)
    #le.inverse_transform([2])
    
    
    #Load model
    with open('/content/drive/MyDrive/Flask/model.json', 'r') as json_file:
        loaded_model_json = json_file.read()
    my_model = model_from_json(loaded_model_json)
    my_model.load_weights('/content/drive/MyDrive/Flask/model_cnnv2.h5')
    
    imagenes_array_train = []
    imagen_array = np.asarray(Image.open(file_path).convert("RGB").resize((176, 208)))       
    # Añadir el array NumPy a la lista
    imagenes_array_train.append(imagen_array)

    img = np.stack(imagenes_array_train)
    
    pred = my_model.predict(img) #Predict                    
    
   

In [4]:
import sys
sys.path.append('/content/drive/My Drive/Flask/')

We'll need to update our path to import from Drive.

In [5]:
import prediction

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install flask_ngrok

In [7]:
!pip install pyngrok==4.1.1

In [8]:
!ngrok authtoken 2ZKF9W8DIq5HiXcXuj4oowsU0zA_5jDHdfeZpumyefsK388mF

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [11]:
from flask import Flask, render_template, request, abort, make_response, jsonify, flash, redirect
from werkzeug.utils import secure_filename
import os
import tempfile
from flask_ngrok import run_with_ngrok

template_folder = "/content/drive/MyDrive/Flask/templates"

app = Flask(__name__, template_folder = template_folder)
run_with_ngrok(app)

app.secret_key = "secret key"
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16 MB
# app.config['UPLOAD_FOLDER'] = "F:/Imagenes/"
app.config['UPLOAD_FOLDER'] = tempfile.gettempdir()

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.errorhandler(413)
def too_large(e):
    return make_response(jsonify(message="File is too large"), 413)

@app.route('/')
def home():
    return render_template('ejemplocopy.html')

@app.route('/upload', methods=['POST', 'GET'])
def upload_file():
    if 'file' in request.files:
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            # Here you should save the file
            # file.save("F:/Imagenes")
            file = file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            # return 'File uploaded successfully'
        try:
            predict = prediction.getPrediction(file_path)
            # Aparece No File uploaded
            flash(predict)
            # return {'Diagnosis is:':predict}
            return redirect('/')
        except Exception as error:
            print("Error:", error)
            return {'error': str(error)}, 500

    return 'No file uploaded'

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5dfd-35-190-176-27.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
